In [1]:
%pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade ipywidgets

Defaulting to user installation because normal site-packages is not writeable
  Using cached ipywidgets-8.0.2-py3-none-any.whl (134 kB)
  Using cached widgetsnbextension-4.0.3-py3-none-any.whl (2.0 MB)
  Using cached jupyterlab_widgets-3.0.3-py3-none-any.whl (384 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [1]:
from sentence_transformers import SentenceTransformer

model_distiluse = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2',device='cuda')




In [2]:
dataset_train_test = pd.read_csv('/home/nikolaypavlychev/DL_course_group/case_2/CompanyNamesSimularity/dataset_train_test.csv',sep=';', dtype=object,
usecols=['pair_id', 'name_1', 'name_2','is_duplicate'])
dataset_train_test.head(10)


,pair_id,name_1,name_2,is_duplicate
0,469360,I B International,Guangzhou City Banger Fu Shoes Co Ltd,0
1,462512,Anchor Logistics,Eastway Logistics Usa Llc,0
2,412189,Fast Rite International Inc,Faj International,0
3,66926,Textiles Omnes SA,Shohagpur Textile Mills Ltd,0
4,219893,Russell Corp Aus PL,Acs International Sa De Cv,0
5,422155,Sika S A,Ultra Plastik,0
6,438135,Samsung India Electronics Pvt Ltd,AG India Retail Pvt Ltd,0
7,349072,Репсол,TERMOPRENE POLYMERS,0
8,172252,Guangzhou Jian Shun Metal Products Co Ltd,Texmate Products Co Ltd,0
9,91962,PROBIGALP,Total Petrochemicals France SA,0


In [3]:
embeddings_name_1 = np.zeros((dataset_train_test.shape[0],512),dtype='float32')
embeddings_name_2 = np.zeros((dataset_train_test.shape[0],512),dtype='float32')

for i,sentence in enumerate(dataset_train_test['name_1']):
    embeddings_name_1[i,:] = model_distiluse.encode(sentence)
for i,sentence in enumerate(dataset_train_test['name_2']):
    embeddings_name_2[i,:] = model_distiluse.encode(sentence)


In [6]:
embeddings_1 = np.hstack((embeddings_name_1,embeddings_name_2))

In [8]:
def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

def simularity(embeddings):
    distances = np.zeros(embeddings.shape[0])

    for i, sentence in enumerate(embeddings):
        vector_i = sentence[0:512]
        vector_j = sentence[512:]
        distances[i] = cos_sim(vector_i,vector_j)

    return distances

cos_distance = simularity(embeddings_1)

dataset_train_test['cos_distance'] = cos_distance

In [9]:
cols = ['emb_'+str(i) for i in range(embeddings_1.shape[1])]
embeddings_1_pd = pd.DataFrame(data=embeddings_1,columns=cols)
embeddings_1_pd.head(10)

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1014,emb_1015,emb_1016,emb_1017,emb_1018,emb_1019,emb_1020,emb_1021,emb_1022,emb_1023
0,-0.010019,0.041886,-0.022464,0.007883,0.032788,-0.038666,-0.041975,-0.013223,-0.010703,-0.014067,...,0.017717,-0.064949,0.033187,-0.036170,0.022240,0.033690,0.026606,0.008227,0.003394,-0.040705
1,-0.001108,0.005430,0.000639,0.022442,0.003073,0.044892,0.008516,-0.050975,-0.046496,0.034313,...,-0.015282,-0.051235,-0.017772,0.039872,0.072017,0.004674,-0.014484,-0.042360,0.018754,0.014960
2,-0.056228,-0.041418,-0.000207,-0.007367,0.032344,-0.097280,-0.070831,-0.095892,0.019264,0.019038,...,0.026453,0.010647,0.043808,0.063090,0.045899,0.078848,-0.054020,0.018797,-0.053563,-0.059737
3,0.009821,0.104008,0.041969,0.038492,0.062954,0.044641,-0.026521,-0.080303,-0.071564,0.048961,...,0.025168,0.101222,-0.035704,0.023215,0.009537,-0.009176,-0.002801,-0.008793,-0.043055,0.001574
4,-0.042139,0.026356,0.116453,-0.024510,0.085993,-0.032767,-0.139877,-0.027299,0.004636,0.005613,...,0.022723,0.009707,0.070634,0.031779,0.060675,0.081432,-0.010950,0.024077,-0.066900,-0.007336
5,0.024450,0.041711,-0.004280,0.013359,0.036126,-0.002998,-0.013572,-0.067998,-0.037822,-0.003677,...,0.038191,0.038397,0.027621,0.058809,-0.087173,-0.049534,0.032280,0.019414,-0.004436,0.004175
6,-0.043863,0.144400,0.049665,-0.002346,0.062322,-0.027895,0.034860,-0.015983,0.022402,0.008212,...,0.021532,-0.010489,-0.028098,0.021536,0.035961,0.025965,0.044225,-0.041691,0.008867,-0.002410
7,0.009519,-0.008982,-0.047006,-0.029204,-0.041061,0.025081,-0.013938,-0.053684,-0.039436,-0.026579,...,0.014573,-0.015408,0.054874,0.019701,-0.064722,-0.015810,0.016742,-0.003004,-0.030361,-0.015998
8,-0.008822,0.075294,0.020024,-0.026539,0.030163,-0.020820,-0.007600,0.007269,0.039772,0.023825,...,-0.025022,0.000999,0.052218,0.004789,0.058940,0.030147,0.046620,-0.003562,0.010777,-0.022225
9,-0.024892,0.011064,-0.012379,0.026645,-0.011526,0.014185,-0.006647,0.020882,0.019070,-0.004773,...,0.006165,-0.035612,0.004891,0.055706,0.031452,0.064064,0.047849,0.051624,-0.061355,-0.016599


In [10]:
dataset_train_test = dataset_train_test[['pair_id', 'name_1', 'name_2','cos_distance','is_duplicate']]
dataset_train_test = pd.concat((dataset_train_test,embeddings_1_pd),axis=1)

dataset_train_test.shape

(7200, 1029)

In [11]:
train, test, y_train, y_test = train_test_split(dataset_train_test.drop(['pair_id', 'name_1', 'name_2','is_duplicate'],axis=1),dataset_train_test['is_duplicate'],
test_size=0.3,shuffle=True,random_state=42)

In [12]:
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression()
logreg.fit(train,y_train)
y_train_pred = logreg.predict(train)

print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      2513
           1       0.91      0.92      0.92      2527

    accuracy                           0.92      5040
   macro avg       0.92      0.92      0.92      5040
weighted avg       0.92      0.92      0.92      5040



In [13]:
y_test_pred = logreg.predict(test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      1087
           1       0.91      0.92      0.92      1073

    accuracy                           0.92      2160
   macro avg       0.92      0.92      0.92      2160
weighted avg       0.92      0.92      0.92      2160



In [4]:
from sentence_transformers import SentenceTransformer

model_miniLM = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',device='cuda')

dataset_train_test = pd.read_csv('/home/nikolaypavlychev/DL_course_group/case_2/CompanyNamesSimularity/dataset_train_test.csv',sep=';', dtype=object,
usecols=['pair_id', 'name_1', 'name_2','is_duplicate'])
dataset_train_test.head(10)

,pair_id,name_1,name_2,is_duplicate
0,469360,I B International,Guangzhou City Banger Fu Shoes Co Ltd,0
1,462512,Anchor Logistics,Eastway Logistics Usa Llc,0
2,412189,Fast Rite International Inc,Faj International,0
3,66926,Textiles Omnes SA,Shohagpur Textile Mills Ltd,0
4,219893,Russell Corp Aus PL,Acs International Sa De Cv,0
5,422155,Sika S A,Ultra Plastik,0
6,438135,Samsung India Electronics Pvt Ltd,AG India Retail Pvt Ltd,0
7,349072,Репсол,TERMOPRENE POLYMERS,0
8,172252,Guangzhou Jian Shun Metal Products Co Ltd,Texmate Products Co Ltd,0
9,91962,PROBIGALP,Total Petrochemicals France SA,0


In [6]:
embeddings_name_1 = np.zeros((dataset_train_test.shape[0],384),dtype='float32')
embeddings_name_2 = np.zeros((dataset_train_test.shape[0],384),dtype='float32')

for i,sentence in enumerate(dataset_train_test['name_1']):
    embeddings_name_1[i,:] = model_miniLM.encode(sentence)
for i,sentence in enumerate(dataset_train_test['name_2']):
    embeddings_name_2[i,:] = model_miniLM.encode(sentence)

In [7]:
embeddings_1 = np.hstack((embeddings_name_1,embeddings_name_2))

def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

def simularity(embeddings):
    distances = np.zeros(embeddings.shape[0])

    for i, sentence in enumerate(embeddings):
        vector_i = sentence[0:384]
        vector_j = sentence[384:]
        distances[i] = cos_sim(vector_i,vector_j)

    return distances

cos_distance = simularity(embeddings_1)

dataset_train_test['cos_distance'] = cos_distance

cols = ['emb_'+str(i) for i in range(embeddings_1.shape[1])]
embeddings_1_pd = pd.DataFrame(data=embeddings_1,columns=cols)

dataset_train_test = dataset_train_test[['pair_id', 'name_1', 'name_2','cos_distance','is_duplicate']]
dataset_train_test = pd.concat((dataset_train_test,embeddings_1_pd),axis=1)

train, test, y_train, y_test = train_test_split(dataset_train_test.drop(['pair_id', 'name_1', 'name_2','is_duplicate'],axis=1),dataset_train_test['is_duplicate'],
test_size=0.3,shuffle=True,random_state=42)

In [9]:
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression()
logreg.fit(train,y_train)
y_train_pred = logreg.predict(train)

print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      2513
           1       0.95      0.95      0.95      2527

    accuracy                           0.95      5040
   macro avg       0.95      0.95      0.95      5040
weighted avg       0.95      0.95      0.95      5040



In [10]:
y_test_pred = logreg.predict(test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1087
           1       0.91      0.93      0.92      1073

    accuracy                           0.92      2160
   macro avg       0.92      0.92      0.92      2160
weighted avg       0.92      0.92      0.92      2160



In [11]:
%pip freeze>>requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
